In [1]:
import chess
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader  

device= torch.device('cuda')
device

device(type='cuda')

In [9]:

class Test_Model(nn.Module):
    def __init__(self):
        super(Test_Model,self).__init__()
        self.conv1 = nn.Conv2d(7,128, kernel_size=3,stride=1)
        self.max = nn.MaxPool2d(2,2)
        self.linear = nn.Sequential(nn.Linear(1152, 512), nn.ReLU(), nn.Linear(512,512), nn.ReLU(), nn.Linear(512,128),
                                   nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,2))
        

    def init_weights(self):
        torch.nn.init.normal_(self.conv1.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv2.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv3.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv4.weight,0, 1e-6)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.max(x)
        x = x.reshape(x.shape[0], -1)
        print(x.shape)
        x = self.linear(x)
        
        
        return x


In [24]:
testing_model = Test_Model().to(device)

testing_model.load_state_dict(torch.load('./previous_models/Prototype2.0-Adam-SGD-1C5L0B-4.pth'))

testing_model.eval()

Test_Model(
  (conv1): Conv2d(7, 128, kernel_size=(3, 3), stride=(1, 1))
  (max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [38]:
from Utils import convert_Fen

net_in = convert_Fen('5rk1/5ppp/p7/1pQ5/4PBb1/3P4/PP5q/R3K1R1 b Q - 2 21').to(device).unsqueeze(0)

e = testing_model(net_in)
F.softmax(e,dim=1)

. . . . . r k .
. . . . . p p p
p . . . . . . .
. p Q . . . . .
. . . . P B b .
. . . P . . . .
P P . . . . . q
R . . . K . R .
torch.Size([1, 1152])


tensor([[0.6351, 0.3649]], device='cuda:0', grad_fn=<SoftmaxBackward0>)